In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy
import matplotlib.mlab as mlab
import scipy.stats
from _helper_functions import tsallis

In [2]:
a = 4.429e-10  # lattice parameter
cellsize = 120
density = 40e-6  # density of spin impurities
# this is very arbitrary
clear_sphere = 0.28e-10  # an area around our atom where we don't allow any impurities

In [3]:
# create Ne crystal lattice
pos = []
for i in range(cellsize+1):
    for j in range(cellsize+1):
        for k in range(cellsize+1):
            pos.append([a*i, a*j, a*k])
for i in range(cellsize+1):
    for j in range(cellsize):
        for k in range(cellsize):
            pos.append([a*i, 0.5*a+a*j, 0.5*a+a*k])
for j in range(cellsize+1):
    for i in range(cellsize):
        for k in range(cellsize):
            pos.append([0.5*a+a*i, a*j, 0.5*a+a*k])
for k in range(cellsize+1):
    for i in range(cellsize):
        for j in range(cellsize):
            pos.append([0.5*a+a*i, 0.5*a+a*j, a*k])
pos = np.asarray(pos)
#pos = pos[pos[:, 2].argsort()]
#pos = pos[pos[:, 1].argsort()]
#pos = pos[pos[:, 0].argsort()]
un = np.sort(np.unique(pos[:, 0]))
val = un[int(len(un)/2)]
idxs = np.where((pos[:, 0] == val) & (pos[:, 1] == val))[0]
atomidx = idxs[int(len(idxs)/2)]

atom = pos[atomidx, :]
pos_cleaned = np.delete(pos, atomidx, axis=0)

In [4]:
len_ne_atoms = len(pos_cleaned)
number_spin_atoms = int(len_ne_atoms*density)
impurity_idx = np.random.choice(range(len_ne_atoms), number_spin_atoms)
pos_cleaned2 = np.delete(pos_cleaned, impurity_idx, axis=0)
impurities = pos_cleaned[impurity_idx, :]

distances = np.linalg.norm((impurities-atom), axis=1)
idxs = np.where(distances < clear_sphere)
impurities = np.delete(impurities, idxs, axis=0)

In [5]:
plt.close('all')
matplotlib.interactive(False)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
#ax.scatter(pos_cleaned2[:, 0], pos_cleaned2[:, 1], pos_cleaned2[:, 2])
ax.scatter(atom[0], atom[1], atom[2])
ax.scatter(impurities[:, 0], impurities[:, 1], impurities[:, 2])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def calculate_dipole_interaction(s1, s2, pos1, pos2):
    """
    s1 is the spin of particle one (for the ground state of Rb85 would be -5/2)
    s2 of particle 2
    pos1, pos2 are the position vectors
    we assume the spins pointing in the z-direction, due to an external magnetic field
    """
    fac = -5.192e-20
    distvec = pos2-pos1
    norm = np.linalg.norm(distvec, axis=1)

    return (s1*distvec[:, 2]*s2*distvec[:, 2]*3-s1*s2)*fac/norm**3

In [13]:
max_magnetic_moment = 5/2

magnetic_moments_rb85 = np.arange(-max_magnetic_moment,
                                  max_magnetic_moment+1/2, 1/2)

realizations = 5000
configurations = 100

shifts = np.zeros((configurations, realizations))
for i in range(configurations):
    impurity_idx = np.random.choice(range(len_ne_atoms), number_spin_atoms)
    impurities = pos_cleaned[impurity_idx, :]

    distances = np.linalg.norm((impurities-atom), axis=1)
    idxs = np.where(distances < clear_sphere)
    impurities = np.delete(impurities, idxs, axis=0)

    for k in range(realizations):
        mag_moments_imp = np.random.choice(
            magnetic_moments_rb85, len(impurities))
        shifts[i, k] = np.sum(calculate_dipole_interaction(-max_magnetic_moment,
                                                           mag_moments_imp, atom, impurities))

In [15]:
plt.close('all')
matplotlib.interactive(False)

fwhms = []
binsum = 0
binrange = np.arange(-30, 30, 0.2)
for shift in shifts:
    (mu, sigma) = scipy.stats.norm.fit(shift*1e-6)
    fwhms.append(2*np.sqrt(2*np.log(2))*sigma)
    n, bins, patches = plt.hist(shift*1e-6, bins=binrange, density=1)

    y = scipy.stats.norm.pdf(bins, mu, sigma)
    binsum += n
    #l = plt.plot(bins, y, 'r--', linewidth=2)

binsum = binsum/len(shifts)

popt, pcov = scipy.optimize.curve_fit(
    tsallis, binrange[:-1], binsum, p0=[1.5, 5])

plt.plot(binrange[:-1], binsum)
plt.plot(binrange[:-1], tsallis(binrange[:-1], *popt))

plt.xlabel("Frequency (MHz)")
plt.ylabel("Occurence")
plt.show()
print("Average FWHM: %.2f MHz" % (popt[1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average FWHM: 4.49 MHz
